In [1]:

%pip install moviepy


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from moviepy.editor import *
import cv2
from fer import FER
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from PIL import Image
import tkinter as tk
from tkinter import filedialog
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# handle file selection, only allow mp4
def get_file_path():
    file_path = filedialog.askopenfilename(
        title="Select Video File",
        filetypes=[("MP4 Files", "*.mp4")]  
    )
    return file_path

# set up a Tkinter root to later display the top k frames 
root = tk.Tk()
root.withdraw() 

#  get the user's video path 
video_path = get_file_path()

segment_length = 10  # break clips into 10 second segments for easier analyzing 
os.makedirs("clips", exist_ok=True)

# load video path to video 
video = VideoFileClip(video_path)
duration = int(video.duration) # get the duration of the video 

# use emtoion detector to detect emotion in actors' faces (use mtcnn = False to decrease runing time)
emotion_detector = FER(mtcnn=False)

# process the first clip 
i = 0
clip = video.subclip(i, min(i + segment_length, duration))
clip_path = f"clips/clip_{i // segment_length}.mp4"

# write the clip to file 
clip.write_videofile(clip_path, codec="libx264", audio_codec="aac", verbose=False, logger=None)



# open video with opencv
clip_video = cv2.VideoCapture(clip_path)
frame_count = 0
emotion_data = []

while True:
    success, frame = clip_video.read()
    if not success:
        break

    # skip frames to decrease running time and also decrease frame similarity 
    if frame_count % 12 != 0:
        frame_count += 1
        continue
    
    # This portion was completed with the assitance with ChatGPT. Its purpose was to guide me on 
    # the FER library to detect emotions and collect metatdata in video frames. 
    # also used in conjunction with https://www.geeksforgeeks.org/facial-expression-recognizer-using-fer-using-deep-neural-net/ for FER details. 
    
    # use emotion detector to find emotion in frames. 
    emotion = emotion_detector.top_emotion(frame)
    if emotion and emotion[1] is not None:

        # create emotion text to display 
        emotion_text = f"{emotion[0]}: {emotion[1]:.2f}"

        # store to use to display later
        emotion_data.append((frame, emotion_text, emotion[1], frame_count))

    frame_count += 1
    
clip_video.release()


#sort the data from highest to lowest 
emotion_sorted = sorted(emotion_data, key=lambda x: x[2], reverse=True)

# only keep the top-k or in this case top 8 frames 
top_8_frames = emotion_sorted[:8]

# setup to display the frames as a popup
num_columns = 4  
num_rows = (len(top_8_frames) // num_columns) + (len(top_8_frames) % num_columns > 0)

fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(num_rows, num_columns, figure=fig)

# display in a grid format as a 2x4 
for idx, (frame, emotion_text, confidence, frame_num) in enumerate(top_8_frames):
    row = idx // num_columns
    col = idx % num_columns

    # conversions 
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    ax = fig.add_subplot(gs[row, col])
    ax.imshow(frame_rgb)
    ax.axis('off')

    # display the metadata at the bottom of the frame 
    ax.set_title(f"Frame {frame_num}: {emotion_text}", fontsize=8)

plt.tight_layout()

popup = tk.Toplevel()
popup.title("Emotion Detection")

# This portion was completed with ChatGPT to create a popup of the final display of the top-k results. 
# using Tkinter
canvas = FigureCanvasTkAgg(fig, master=popup)  
canvas.draw()
canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

popup.mainloop()

